In [1]:
import json

In [2]:
pentaminoes_str = '''
.*.
***
.*.

*
*
*
*
*

***
*.*

..*
.**
**.

*..
*..
***

*..
***
..*

*..
***
*..

**.
.**
.*.

****
..*.

.*
.*
**
*.

.*
**
**

...*
****
'''

In [3]:
pentaminoes = [p.strip() for p in pentaminoes_str.split('\n\n')]

In [4]:
def flip_vertical(p):
    return '\n'.join(p.split('\n')[::-1])

def flip_horizontal(p):
    return '\n'.join(s[::-1] for s in p.split('\n'))

def transpose(p):
    toks = p.split('\n')
    return '\n'.join(
        ''.join(tok[i] for tok in toks) 
        for i in range(len(toks[0]))
    )

def flip_flop(p):
    funcs = [
        flip_vertical,
        flip_horizontal,
        transpose
    ]
    s = set()
    for i in range(8):
        result = p
        for j in range(3):
            if (i & (1 << j)) != 0:
                result = funcs[j](result)
        s.add(result)
    return s

In [5]:
def dims(p):
    toks = p.split('\n')
    return len(toks), len(toks[0])

def shift(p, sr, sc):
    toks = p.split('\n')
    r, c = len(toks), len(toks[0])
    result = []
    for ir in range(8):
        if ir < sr or ir >= sr + r:
            result.append('.' * 8)
        else:
            result.append(
                '.' * sc +
                toks[ir - sr] +
                '.' * (8 - sc - len(toks[ir - sr]))
            )
    return '\n'.join(result)

def all_shifts(p):
    r, c = dims(p)
    result = []
    for ir in range(8 - r + 1):
        for ic in range(8 - c + 1):
            result.append(shift(p, ir, ic))
    return result

In [6]:
test = '''
.....*..
.*......
..*.....
..*.....
........
........
........
........
'''.strip()

def to_bin(field):
    toks = field.strip().split('\n')
    result = 0
    for tok in toks:
        result <<= 8
        for i in range(len(tok)):
            if tok[i] == '*':
                result |= 1 << i
    return result

def from_bin(value, sub = '*'):
    result = []
    for i in range(8):
        tok_value = ((value >> (i * 8)) & 255)
        result.append(''.join(
            sub if (tok_value & (1 << j)) != 0 else '.' 
            for j in range(8)
        ))
    return '\n'.join(result[::-1])

test == from_bin(to_bin(test))

True

In [7]:
def all_pentaminoes_bin():
    result = []
    for i in range(len(pentaminoes)):
        p = pentaminoes[i]
        sub_result = []
        for sub_p in flip_flop(p):
            for sub_shift in all_shifts(sub_p):
                sub_result.append(to_bin(sub_shift))
        result.append(sub_result)
    return result

pentaminoes_bin = all_pentaminoes_bin()

In [8]:
def all_bin_per_cell():
    result = {}
    for cell in range(64):
        result[cell] = {}
        for i in range(len(pentaminoes_bin)):
            result[cell][i] = []
            for bin_val in pentaminoes_bin[i]:
                if (bin_val & (1 << cell)) != 0:
                    result[cell][i].append(bin_val)
    return result

bin_per_cell = all_bin_per_cell()

In [20]:
def task_month(month):
    month -= 1
    return shift('**', month // 4, 2 * (month % 4))

def task_day(day):
    if day == 31:
        return shift('*', 7, 7)
    if day <= 24:
        day -= 1
        return shift('*', 3 + day // 8, day % 8)
    else:
        return shift('*', 6, day - 25 + 1)

def task_weekday(day):
    day = 0 if day == 7 else day
    return shift('*', 7, day)
    
def task(month, day, weekday):
    value = to_bin(task_month(month)) | to_bin(task_day(day)) | to_bin(task_weekday(weekday))
    return value

In [18]:
def task_set():
    result = {}
    for m in range(1, 12 + 1):
        for d in range(1, 31 + 1):
            for wd in range(1, 7 + 1):
                result['{}/{}/{}'.format(m, d, wd)] = task(m, d, wd)
    return result

tasks = task_set()

In [19]:
def write_tasks(fpath, tasks_to_store):
    with open(fpath, 'w') as f:
        for k in tasks_to_store:
            f.write('{} {}\n'.format(k, tasks_to_store[k]))
            
write_tasks("cruncher/task.data", tasks)

In [12]:
def write_masks(fpath):
    with open(fpath, 'w') as f:
        for c in range(64):
            for p in range(12):
                masks = bin_per_cell[c][p]
                f.write('{} {}\n'.format(
                    len(masks),
                    " ".join(str(mask) for mask in masks)
                ))
                
write_masks("cruncher/cpm.data")

In [13]:
output_test = "2 16185937060769562624 10 505247583195627520 7 1732768752723820544 0 18260963992010752 6 4521432331583488 9 4415428231168 5 12918849536 8 551915913216 4 1880100864 11 69376 3 2121920 1 62"

def combine(masks):
    base = from_bin(0)
    result = []
    for i in range(len(base)):
        if base[i] != '.':
            result.append(base[i])
            continue;
        cand = base[i];
        for mask in masks:
            if mask[i] != '.':
                cand = mask[i]
        result.append(cand)
    return ''.join(result)
        

def parse_output(output):
    toks = output.split(' ')
    result = {chr(int(xs) + ord('a')): int(ys) for xs, ys in zip(toks[0::2], toks[1::2])}
    masks = [from_bin(result[key], key) for key in result]
    return combine(masks)

In [65]:
def parse_solutions(sol_file):
    with open(sol_file, 'r') as f:
        lines = [l.strip() for l in f.readlines() if l.strip()]

    result = {}
    for l in lines:
        toks = l.split(' ')
        date = toks[0]
        pattern = from_bin(int(toks[1]))
        tesselation = parse_output(' '.join(toks[2:]))
        result[date] = {
            'pattern': pattern,
            'tessellation': tesselation,
        }
    return result

solutions = parse_solutions('cruncher/solutions_standard.data')

In [67]:
print(solutions['4/2/5']['pattern'])
print()
print(solutions['4/2/5']['tessellation'])

......**
........
........
.*......
........
........
........
.....*..

efiiii..
efffiall
eeefaaal
b.jjjagl
bjjhgggl
bcchhdgk
bchhddkk
bccdd.kk


In [68]:
with open('tessellation.json', 'w') as f:
    f.write(json.dumps(solutions))

In [69]:
with open('tessellation.txt', 'w') as f:
    for k in tasks:
        f.write('{}\n{}\n\n'.format(k, solutions[k]['tessellation']))

In [44]:
def task_year_extended(year):
    year -= 2021
    return shift('*', year // 4, year % 4)

def task_month_extended(month):
    month -= 1
    return shift('*', month // 4, 4 + month % 4)

def task_extended(year, month, day, weekday):
    return \
        to_bin(task_year_extended(year)) | \
        to_bin(task_month_extended(month)) | \
        to_bin(task_day(day)) | \
        to_bin(task_weekday(weekday))

In [45]:
def task_set_extended():
    result = {}
    for y in range(2021, 2032 + 1):
        for m in range(1, 12 + 1):
            for d in range(1, 31 + 1):
                for wd in range(1, 7 + 1):
                    result['{}/{}/{}/{}'.format(y, m, d, wd)] = task_extended(y, m, d, wd)
    return result

tasks_extended = task_set_extended()

In [46]:
write_tasks("cruncher/task_extended.data", tasks_extended)

In [48]:
solutions_extended = parse_solutions('cruncher/solutions_extended.data')

In [62]:
print(solutions_extended['2022/3/2/5']['pattern'])
print()
print(solutions_extended['2022/3/2/5']['tessellation'])

.*....*.
........
........
.*......
........
........
........
.....*..

k.hjjj.e
kkhhhjje
kkghaeee
b.gaaadd
bgggaddl
biiiidfl
bcicfffl
bcccf.ll


In [64]:
with open('tessellation_extended.json', 'w') as f:
    f.write(json.dumps(solutions_extended))

with open('tessellation_extended.txt', 'w') as f:
    for k in tasks_extended:
        f.write('{}\n{}\n\n'.format(k, solutions_extended[k]['tessellation']))